In [1]:
import os
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime as dt 
import pandas_profiling
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [31]:
pwd = os.getcwd()
print(pwd)

data_Train = pd.read_csv('C:\\Users\\IQBALE\\ClassRoom\\Hackathons\\ChartbustersParticipantsData\\Data_Train.csv')
data_Test = pd.read_csv('C:\\Users\\IQBALE\\ClassRoom\\Hackathons\\ChartbustersParticipantsData\\Data_Test.csv')

C:\Users\IQBALE\ClassRoom\Hackathons


In [32]:
data_Test.shape
data_Train.shape
data_Train.describe()
data= pd.concat([data_Train,data_Test],axis=0)
data.shape
data.isna().sum() 
data.duplicated().sum() # 0 
data[pd.isnull(data["Song_Name"])]
data.shape
data = data[data.isna()['Song_Name'] == False]
data.isna().sum()
data["Date"] = pd.to_datetime(data["Timestamp"])
data["Year"] = data["Date"].dt.year
data["Year"] = 2000 - data["Year"]
data["Year"].unique()
data["Comments"].unique() # number values 
data["Country"].unique()
data["Followers"].unique()
data["Genre"].unique()
data["Likes"].unique()
data["Views"].isna().sum()

19615

In [33]:
data.isna().sum()

Comments          0
Country           0
Followers         0
Genre             0
Likes             0
Name              0
Popularity        0
Song_Name         0
Timestamp         0
Unique_ID         0
Views         19615
Date              0
Year              0
dtype: int64

In [34]:
data.head()

,Comments,Country,Followers,Genre,Likes,Name,Popularity,Song_Name,Timestamp,Unique_ID,Views,Date,Year
0,4,AU,119563,danceedm,499,Hardstyle,97,N-Vitral presents BOMBSQUAD - Poison Spitter (...,2018-03-30 15:24:45.000000,413890,14017.0,2018-03-30 15:24:45,-18
1,17,AU,2141,danceedm,49,Dj Aladdin,17,Dj Aladdin - Old School Hip Hop Quick Mix,2016-06-20 05:58:52.000000,249453,1918.0,2016-06-20 05:58:52,-16
2,11,AU,22248,danceedm,312,Maxximize On Air,91,Maxximize On Air - Mixed by Blasterjaxx - Epis...,2015-05-08 17:45:59.000000,681116,9668.0,2015-05-08 17:45:59,-15
3,2,AU,393655,rbsoul,"2,400",GR6 EXPLODE,76,MC Yago - Tenho Compromisso (DJ R7),2017-06-08 23:50:03.000000,387253,113036.0,2017-06-08 23:50:03,-17
4,81,AU,201030,danceedm,"3,031",Tritonal,699,Escape (feat. Steph Jones),2016-09-17 20:50:19.000000,1428029,110024.0,2016-09-17 20:50:19,-16


In [35]:
data_input = data[['Comments', 'Followers', 'Genre', 'Likes', 
       'Popularity', 'Views', 'Year']]
data_input.dtypes
data_input["Likes"] = data_input.Likes.str.replace(',','')
data_input["Popularity"] = data_input.Popularity.str.replace(',','')

"""if data_input["Likes"][-1] == 'K':
    data_input["Likes"] = float(data_input["Likes"][:-1]) * 1000
else if data_input["Likes"][-1] == 'M':
    data_input["Likes"] = float(data_input["Likes"][:-1]) * 1000000
else:
    data_input["Likes"] = float(data_input["Likes"])"""

def convert_num(x):
    if x[-1] == 'K':
        return float(x[:-1]) * 1000
    elif x[-1] == 'M':
        return float(x[:-1]) * 1000000
    else: 
        return float(x)

data_input["Likes"] = data_input["Likes"].apply(convert_num)
data_input["Popularity"] = data_input["Popularity"].apply(convert_num)

le = preprocessing.LabelEncoder()
data_input["Genre"] = le.fit_transform(data_input["Genre"])

Train_input = data_input[data_input.isna()['Views'] == False] 
Test_input = data_input[data_input.isna()['Views'] == True]

#     print(df[i])

In [23]:
Train_input.shape

(78457, 7)

In [24]:
Test_input.shape

(19615, 7)

In [25]:
Test_input.head()

,Comments,Followers,Genre,Likes,Popularity,Views,Year
0,26,2568,0,5428.0,118.0,NaN,-19
1,18,155645,5,738.0,195.0,NaN,-15
2,18,29446,5,386.0,96.0,NaN,-13
3,1,799410,0,388.0,19.0,NaN,-7
4,0,2,19,79.0,1.0,NaN,-18


In [37]:
X_Train = Train_input.drop(columns=['Views'])
Y_Train = Train_input[['Views']]
X_Test  = Test_input.drop(columns=['Views'])

scaler = StandardScaler()
X_Train = scaler.fit_transform(X_Train)
X_Test  = scaler.fit(X_Test)


In [39]:
X_Train[:5]

array([[-0.15355164, -0.30750591, -0.62976993, -0.1577008 , -0.17126216,
        -0.74654659],
       [-0.13561104, -0.40802812, -0.62976993, -0.16622767, -0.18789289,
        -0.02456096],
       [-0.14389132, -0.39081499, -0.62976993, -0.16124419, -0.17250947,
         0.33643185],
       [-0.15631174, -0.07286222,  1.38522503, -0.12167953, -0.17562773,
        -0.38555378],
       [-0.04728806, -0.23776394, -0.62976993, -0.10972297, -0.04611596,
        -0.02456096]])

In [20]:
data_input["Genre"].unique()

array([ 5, 17,  0, 20, 14, 10,  6, 12,  3,  2, 16, 19, 18,  4,  7,  1,  9,
        8, 15, 11, 13], dtype=int64)

In [17]:
Y_train_train.head()

,Views
22235,2438.0
63524,926.0
11152,668780.0
28569,418.0
41403,448.0


In [44]:
X_train_train, X_train_test, Y_train_train, Y_train_test = train_test_split(X_Train,Y_Train,test_size=0.3,random_state= 8)

In [49]:
def model_fit(model,X_train_train,X_train_test,Y_train_train,Y_train_test,X_test):   
    model.fit(X_train_train,Y_train_train)
    #pd.DataFrame(model.coef_,X_train_train.columns).plot(kind="bar")
    train_predict = model.predict(X_train_train)
    test_predict = model.predict(X_train_test)
    print("Model:" , model)
    print("Train RMSE : ",np.sqrt(mean_squared_error(train_predict,Y_train_train)))
    print("Test RMSE : ",np.sqrt(mean_squared_error(test_predict,Y_train_test)))
    print ( 'number of features used: ' ,  np.sum(model.coef_!=0) )    
    test_hack_predict = model.predict(test_X.drop(columns=["Item_Identifier","Outlet_Identifier"]))
    test_pred_df = pd.DataFrame(test_predict,columns= ["Item_Outlet_Sales"])
    result = pd.concat([test_X[["Item_Identifier", "Outlet_Identifier"]],test_pred_df] ,axis = 1)
    result.to_csv(filename,index = False)

In [48]:
X_train_test[:5]

array([[-0.15769178, -0.28995806,  1.38522503, -0.16461704, -0.18851654,
        -1.1075394 ],
       [-0.14389132, -0.34967966,  1.72105752, -0.14303459, -0.19142692,
         1.78040309],
       [-0.04866811,  0.06871484, -0.62976993, -0.13058537, -0.02886158,
        -0.02456096],
       [-0.15907183, -0.40937644,  0.88147629, -0.16708035, -0.19142692,
        -0.38555378],
       [-0.15769178, -0.40460895, -1.46935116, -0.12973268, -0.16544141,
        -0.38555378]])

In [50]:
model = LinearRegression() 
model_fit(model,X_train_train,X_train_test,Y_train_train,Y_train_test,X_Test)

Model: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Train RMSE :  710344.1912112432
Test RMSE :  767957.5616918649
number of features used:  6


In [54]:
model = Lasso(alpha=0.5)
model_fit(model,X_train_train,X_train_test,Y_train_train,Y_train_test,X_Test)

Model: Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Train RMSE :  710344.1912142438
Test RMSE :  767957.4554779326
number of features used:  6
